# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

# Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The API, however, provides you with a lot of information that will not be pertinent to your analysis. You will pull data from the API and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the API will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

# Part 2 - Create ETL pipeline for the business data from the API

## Details

Now that you know what data you need from the API, you want to write code that will execute an API call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*API call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

## Request

### Imports and Setup

In [9]:
import requests
import pandas as pd
import json
import csv
import numpy as np

with open(r'/Users/jax/documents/Flatiron/.secrets/creds.json') as f:
    keys = json.load(f)

yelp_client_id = keys['id']
yelp_key = keys['key']

### ƒ: yelp_request

 - Params: search term (eg. "wineries); location; yelp_key variable (from Imports); and changing setting to print details

In [45]:
def yelp_request(term, location, yelp_key, verbose=True):
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50,
                    'categories': 'Winery[CA]',
                    'offset': 50
                }
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

### Sending the request and saving the response

- Un-comment the next line to run the response

In [46]:
response = yelp_request('winery','Napa Valley', yelp_key)
response.keys()

<Response [200]>
<class 'str'>
{"businesses": [{"id": "tlP1ir-OEzsAYNWJYYNMNg", "alias": "goosecross-cellars-yountville", "name": "Goosecross Cellars", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/r0J3WbihTSMSsYKQo-73kA/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/goosecross-cellars-yountville?adjust_creative=16wS7VfcBVJb7aiWJZ95fg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=16wS7VfcBVJb7aiWJZ95fg", "review_count": 387, "categories": [{"alias": "wineries", "title": "Wineries"}], "rating": 4.5, "coordinates": {"latitude": 38.4266242980957, "longitude": -122.352890014648}, "transactions": [], "price": "$$", "location": {"address1": "1119 State Ln", "address2": "", "address3": "", "city": "Yountville", "zip_code": "94599", "country": "US", "state": "CA", "display_address": ["1119 State Ln", "Yountville, CA 94599"]}, "phone": "+17079441986", "display_phone": "(707) 944-1986", "distance": 14497.879120686433}, {"id": "oMOYzH6r2oPeplABN1ayDg"

dict_keys(['businesses', 'total', 'region'])

#### Saving/Loading as JSON for simplicity while iterating

In [17]:
# try:
#     with open(r'data\response.txt', 'w') as f:
#         json.dump(response, f)
# except IOError:
#     print("I/O error")

In [47]:
with open('data/response.txt') as json_file:
    raw_data = json.load(json_file)

### Identifying and Exploring Keys

In [48]:
# Identify keys

print(response.keys())

dict_keys(['businesses', 'total', 'region'])


#### Exploring the "Businesses" Key

In [49]:
response['total']

1100

In [50]:
response['businesses'][0]

{'id': 'tlP1ir-OEzsAYNWJYYNMNg',
 'alias': 'goosecross-cellars-yountville',
 'name': 'Goosecross Cellars',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/r0J3WbihTSMSsYKQo-73kA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/goosecross-cellars-yountville?adjust_creative=16wS7VfcBVJb7aiWJZ95fg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=16wS7VfcBVJb7aiWJZ95fg',
 'review_count': 387,
 'categories': [{'alias': 'wineries', 'title': 'Wineries'}],
 'rating': 4.5,
 'coordinates': {'latitude': 38.4266242980957, 'longitude': -122.352890014648},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '1119 State Ln',
  'address2': '',
  'address3': '',
  'city': 'Yountville',
  'zip_code': '94599',
  'country': 'US',
  'state': 'CA',
  'display_address': ['1119 State Ln', 'Yountville, CA 94599']},
 'phone': '+17079441986',
 'display_phone': '(707) 944-1986',
 'distance': 14497.879120686433}

In [51]:
response['businesses'][0]['categories'][0]['alias']

'wineries'

In [52]:
response['businesses'][0]['categories'][0]['title']

'Wineries'

#### Exploring the "Total" Key

In [53]:
response['total']

# How many businesses are there in total for my request?

1100

#### Exploring the "Region" Key

In [54]:
response['region']

# From which geographical area will my results come?

{'center': {'longitude': -122.29293823242188, 'latitude': 38.30551908482599}}

## Parse

In [55]:
response.keys()

dict_keys(['businesses', 'total', 'region'])

### ƒ: parse_data

In [56]:
def parse_data(list_of_data):
    
    data_dict = []
    
    # Loop through each business in the list of businesses
    # Add specific k:v pairs to a dictionary
    # These pairs will be used to build a DF afterwards
    
    for business in list_of_data:
        if 'price' not in business:
            business['price'] = np.nan
            
            # Verify that the "price" key is in the selected business dict
            
        details = {'name': business['name'],
                     'location': ' '.join(business['location']['display_address']),
                     'id': business['id'],
                     #'categories': business['categories'],
                     'alias': business['categories'][0]['alias'],
                     'title': business['categories'][0]['title'],
                     'rating': business['rating'],
                     'review_count': business['review_count'],
                     'price': business['price'],
                     'latitude': business['coordinates']['latitude'],
                     'longitude': business['coordinates']['longitude']
                    }

        # If the "price" key is missing, then skip adding that key
        # This avoids an error when getting the desired info
        
#         else:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
        
        # Add the new dictionary to the previous list
        
        data_dict.append(details)
    
    # Adjust the 'location' value to be a single string (not 2 str in a list)
    
#     for biz in parsed_data:
#         biz['location'] = ' '.join(biz['location'])
        
    # Create a DataFrame from the resulting list
    
    norcaldata_df = pd.DataFrame(data_dict)
    
#     df_parsed_data.dropna(inplace=True)
    
    return norcaldata_df

In [57]:
parsed_results = parse_data(response['businesses'])

In [58]:
parsed_results['title']

0              Wineries
1               Museums
2     Wine Tasting Room
3              Wineries
4              Wineries
5              Wineries
6              Wineries
7              Wineries
8              Wineries
9              Wineries
10    Wine Tasting Room
11           Wine Tours
12             Wineries
13             Wineries
14             Wineries
15             Wineries
16             Wineries
17             Wineries
18             Wineries
19             Wineries
20             Wineries
21             Wineries
22             Wineries
23       American (New)
24             Wineries
25     Horseback Riding
26    Wine Tasting Room
27             Wineries
28             Wineries
29    Wine Tasting Room
30             Wineries
31                Limos
32    Wine Tasting Room
33           Wine Tours
34    Wine Tasting Room
35             Wineries
36             Wineries
37             Wineries
38    Wine Tasting Room
39             Wineries
40           Wine Tours
41             W

In [59]:
# Identify totals of NaN values

parsed_results.isnull().sum()

name             0
location         0
id               0
alias            0
title            0
rating           0
review_count     0
price           13
latitude         0
longitude        0
dtype: int64

In [60]:
# Simple statistical exploration of quantitative values

parsed_results.describe()

,rating,review_count,latitude,longitude
count,50.000000,50.00000,50.000000,50.000000
mean,4.590000,216.32000,38.325258,-122.323162
std,0.412682,351.15747,0.065814,0.053924
min,3.500000,1.00000,38.214017,-122.457671
25%,4.500000,18.00000,38.275199,-122.354105
50%,4.500000,65.50000,38.310754,-122.310122
75%,5.000000,234.25000,38.380386,-122.286983
max,5.000000,1449.00000,38.447904,-122.235818


## Updating Requests for Pagination

### ƒ: yelp_request_offset

In [80]:
def yelp_request_offset(term, location, yelp_key, offset=0, verbose=False):
    
    url = 'https://api.yelp.com/v3/businesses/search'
    offset = 0
    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50,
                    'categories': 'Winery',
                    'offset': offset
                        }
    
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

#### Test 1

In [97]:
test1 = yelp_request_offset('winery', 'Napa Valley', yelp_key, offset=0, verbose=False)
test1

{'businesses': [{'id': 'mO8n3zTLoFhlmcfQr7X_TQ',
   'alias': 'hendry-vineyard-and-winery-napa',
   'name': 'Hendry Vineyard and Winery',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/3C-9EJiUU1qTtv0NazPB6g/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/hendry-vineyard-and-winery-napa?adjust_creative=16wS7VfcBVJb7aiWJZ95fg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=16wS7VfcBVJb7aiWJZ95fg',
   'review_count': 658,
   'categories': [{'alias': 'wineries', 'title': 'Wineries'}],
   'rating': 5.0,
   'coordinates': {'latitude': 38.32168, 'longitude': -122.34481},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '3104 Redwood Rd',
    'address2': None,
    'address3': '',
    'city': 'Napa',
    'zip_code': '94558',
    'country': 'US',
    'state': 'CA',
    'display_address': ['3104 Redwood Rd', 'Napa, CA 94558']},
   'phone': '+17072268320',
   'display_phone': '(707) 226-8320',
   'distance': 4753.3197480330155}

In [98]:
test1.keys()

dict_keys(['businesses', 'total', 'region'])

In [99]:
test1['total']

1100

In [100]:
len(test1['businesses'])

50

#### Test 2

In [101]:
test2 = yelp_request_offset('Winery', 'Napa Valley', yelp_key, offset=50, verbose=False)
test2

{'businesses': [{'id': 'mO8n3zTLoFhlmcfQr7X_TQ',
   'alias': 'hendry-vineyard-and-winery-napa',
   'name': 'Hendry Vineyard and Winery',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/3C-9EJiUU1qTtv0NazPB6g/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/hendry-vineyard-and-winery-napa?adjust_creative=16wS7VfcBVJb7aiWJZ95fg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=16wS7VfcBVJb7aiWJZ95fg',
   'review_count': 658,
   'categories': [{'alias': 'wineries', 'title': 'Wineries'}],
   'rating': 5.0,
   'coordinates': {'latitude': 38.32168, 'longitude': -122.34481},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '3104 Redwood Rd',
    'address2': None,
    'address3': '',
    'city': 'Napa',
    'zip_code': '94558',
    'country': 'US',
    'state': 'CA',
    'display_address': ['3104 Redwood Rd', 'Napa, CA 94558']},
   'phone': '+17072268320',
   'display_phone': '(707) 226-8320',
   'distance': 4753.3197480330155}

#### Loop

In [102]:
results = yelp_request_offset('Winery', 'Napa Valley', yelp_key, offset=0, verbose=False)

num_pages = results['total']//50+1

print(results['total'])
print(len(results['businesses']))
print(num_pages)

1100
50
23


In [103]:
cur = 50
parsed_results_dfs = []

for num in range(num_pages):
    results = yelp_request_offset('Winery', 'Napa Valley', yelp_key, offset=cur, verbose=False)
    parsed_results = parse_data(results['businesses'])
    parsed_results_dfs.append(parsed_results)
    cur += 50
    

In [104]:
parsed_results_dfs

[                                                name  \
 0                         Hendry Vineyard and Winery   
 1                                   Domaine Carneros   
 2                          Artesa Vineyards & Winery   
 3                                Bouchaine Vineyards   
 4                                    Paraduxx Winery   
 5                                      Jarvis Winery   
 6                               Vinoce & Twenty Rows   
 7                               White Rock Vineyards   
 8                              Cuvaison Estate Wines   
 9                    St Clair Brown Winery & Brewery   
 10                                       Clos du Val   
 11                        William Hill Estate Winery   
 12                          Shadybrook Estate Winery   
 13                               Platypus Wine Tours   
 14                                  Ashes & Diamonds   
 15                          Stag's Leap Wine Cellars   
 16                            

In [92]:
df_concat = pd.concat(parsed_results_dfs, ignore_index=True)
df_concat

,name,location,id,alias,title,rating,review_count,price,latitude,longitude
0,Hendry Vineyard and Winery,"3104 Redwood Rd Napa, CA 94558",mO8n3zTLoFhlmcfQr7X_TQ,wineries,Wineries,5.0,658,$$,38.321680,-122.344810
1,Domaine Carneros,"1240 Duhig Rd Napa, CA 94559",8eGTOeEQpUpYb89ISug3ag,wineries,Wineries,4.0,2239,$$,38.255534,-122.351391
2,Artesa Vineyards & Winery,"1345 Henry Rd Napa, CA 94558",SSvGGG4YELueu8SF3mE_3g,winetastingroom,Wine Tasting Room,4.0,1506,$$,38.269160,-122.355367
3,Bouchaine Vineyards,"1075 Buchli Station Rd Napa, CA 94559",de7QOXpKlgpaYbUK9bA-Hg,winetastingroom,Wine Tasting Room,4.5,184,$$,38.226197,-122.331614
4,Paraduxx Winery,"7257 Silverado Trl Napa, CA 94558",cBFZALrZbLV5XBsiPcgknQ,wineries,Wineries,4.5,373,$$,38.435480,-122.351430
...,...,...,...,...,...,...,...,...,...,...
1145,Stonehedge Winery,"1004 Clinton St Napa, CA 94559",obZCSR6g1Tp0ok207pCULA,wine_bars,Wine Bars,4.5,138,$$,38.301341,-122.287519
1146,Etude Wines,"1250 Cuttings Wharf Rd Napa, CA 94559",H_0SOBdmEzzkk9398319TA,wineries,Wineries,4.0,176,$$,38.245847,-122.315023
1147,Eleven Eleven Winery,"620 Trancas St Napa, CA 94558",j8TEOilvSUqT2okhmzO9hA,wineries,Wineries,5.0,36,$$$,38.324989,-122.286911
1148,JaM Cellars,"1460 First St Napa, CA 94559",z31nB1cGHiwJ3MoS9qwEMQ,wineries,Wineries,4.5,163,$$,38.297435,-122.289343


## Save

### ƒ: df_save

Save parsed results to a .csv file

In [107]:
df_concat.shape

(1150, 10)

In [112]:
df_concat.to_csv('data\all_ca_wineries.csv', mode = a, header = False)

NameError: name 'a' is not defined

In [111]:
df = pd.read_csv("data/all_ca_wineries.csv", index_col=0)
df

,name,location,id,alias,title,rating,review_count,price,latitude,longitude
0,The Winery Restaurant & Wine Bar,"4301 La Jolla Village Dr Ste 2040 San Diego, C...",76ADW8x8J_69qbtsc5F-2g,bars,Bars,4.0,492,$$,32.872428,-117.213775
1,Baja Winery Tours,"4629 Cass St San Diego, CA 92109",vVaNDvLrCCE_Cw_DyPnBpA,winetours,Wine Tours,5.0,66,NaN,32.798916,-117.252111
2,Bernardo Winery,"13330 Paseo Del Verano Norte San Diego, CA 92128",DknnpiG1p4OoM1maFshzXA,winetastingroom,Wine Tasting Room,4.5,626,$$,33.032800,-117.046460
3,Callaway Vineyard & Winery,"517 4th Ave Ste 101 San Diego, CA 92101",Cn2_bpTngghYW1ej4zreZg,winetastingroom,Wine Tasting Room,5.0,100,$$,32.710751,-117.160918
4,Négociant Winery,"2419 El Cajon Blvd San Diego, CA 92104",Cc1sQWRWgGyMCjzX2mmMQQ,winetastingroom,Wine Tasting Room,4.5,103,$$,32.754880,-117.138280
...,...,...,...,...,...,...,...,...,...,...
192,San Diego Limobuses,"3333 Midway Dr Ste 206 San Diego, CA 92110",SCaFGyzrTGTI6aQHhLxbgA,limos,Limos,3.0,46,NaN,32.750060,-117.211380
193,Alpine Discount Liquor,"2223 Alpine Blvd Alpine, CA 91901",-ARx5ShNxJgjyahKnikTnA,beer_and_wine,"Beer, Wine & Spirits",3.5,3,$,32.835287,-116.765910
194,Village Wine & Spirits,"1552 Encinitas Blvd Encinitas, CA 92024",XkGnb-YxP5MK_ok1X011RA,beer_and_wine,"Beer, Wine & Spirits",4.0,24,$$,33.045896,-117.255584
195,The Destination Wedding Group,"Escondido, CA 92033",lJwxe_fjdt-e8xPrDq63fA,wedding_planning,Wedding Planning,5.0,18,NaN,33.123470,-117.086520


#### Old Stuff

In [ ]:
# from helper_funcs import *

In [ ]:


# # create a variable  to keep track of which result you are in. 
# cur = 0

# #set up a while loop to go through and grab the result 
# while cur < num and cur < 1000:
#     #set the offset parameter to be where you currently are in the results 

#     #make your API call with the new offset number
#     results = yelp_call_offset('winery', 'San Diego', yelp_key, offset=cur, verbose=False)
    
#     #after you get your results you can now use your function to parse those results
#     parsed_results = parse_results(results)
    
#     # use your function to insert your parsed results into the db
#     df_save(parsed_results)
#     #increment the counter by 50 to move on to the next results
#     cur += 50

## Edit and Condense

Goal: condense the details down to specific functions (perhaps one function?) to pull, clean, and save data.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

# ƒ: GET FULL DATA

In [ ]:
# results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)

# num_pages = results['total']//50+1

# print(results['total'])
# print(len(results['businesses']))
# print(num_pages)

In [ ]:
def get_full_data(term, location, yelp_key, file_name = 'data/wineries.csv'):
    '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    blank_df = pd.DataFrame()
    blank_df.to_csv(file_name)
    
    # Process first request to Yelp API and calculate number of pages 
    results = yelp_request_offset(term, location, yelp_key, offset=0, verbose=False)
    num_pages = results['total']//50+1
    
    # Print out confirmation feedback
    print(f'For {term} and {location}: ')
    print(f"    Total number of results: {results['total']}.")
    print(f'    Total number of pages: {num_pages}.')
    
    # Create offset for further results and create empty list
    cur = 0
    parsed_results_dfs = []

    # Retrieves remaining pages
    for num in range(num_pages-1):
        try:
            results = yelp_request_offset(term, location, yelp_key, offset=cur, verbose=False)
            parsed_results = parse_data(results['businesses'])
            parsed_results_dfs.append(parsed_results)
            cur += 50
        except:
            print(f'Error on page {num}.')
            parsed_results_dfs.to_csv(file_name, mode='a')

    # Concatenate DataFrames and save to .csv
    df_concat = pd.concat(parsed_results_dfs, ignore_index=True)

    try:
        df_concat.to_csv(file_name)
        print(f'Saved to {file_name}.')
    except:
        print(f'Error, did not save.')
        
    return df_concat

In [ ]:
df2 = get_full_data('winery', 'San Diego', yelp_key)
df2

In [ ]:
# print(df2['name'].to_string())

In [ ]:
# df3 = get_full_data('pizza', 'Baltimore', yelp_key)

In [ ]:
# df3

## Cleaning Data

In [ ]:
# df2.isna().sum()

In [ ]:
# df2[df2['price'].isna()]

## Identifying top 3 aliases

In [ ]:
# Create new DF showing only alias and title columns
df2_alias = df2.loc[:,['alias', 'title']]

# Identify top 2 aliases 
df2_alias_count = df2_alias.groupby('alias').count().sort_values(['title'],\
                                                        ascending=False)[:2]
df2_alias_count.reset_index(inplace=True)

print(df2_alias_count)

# display them as a list
aliases_top_3 = df2_alias_count['alias'].tolist()
print(aliases_top_3)

# Note: initially tried top 3, but it returned distributors, not wineries

In [ ]:
# Selecting rows based on condition

df3 = df2[df2['alias'].isin(aliases_top_3)]
df3

In [ ]:
df3.to_csv('data/filtered_aliases.csv')

In [ ]:
df_saved = pd.read_csv("data/filtered_aliases.csv", index_col=0)
df_saved

### Ignore - old code

In [ ]:
# df2_alias_sorted = df2_alias_count.sort_values(['title'], ascending=False)
# df3_alias = pd.DataFrame(df2_alias_sorted)
# df3_alias

# df3_alias.groupby('alias').sum().sort_values(['title'], ascending=False)[:3]

# df_alias_top_3 = df3_alias.loc[:, 'title']#[:3]
# df_alias_top_3

In [ ]:
# df2_alias = df2.loc[:,'alias']

# df2_alias_count = df2.groupby('alias').count()#.sort_values(['alias'], ascending=False)#[:3]
# df2_alias_count.reset_index(inplace=True)

# df2_alias_count

# Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you have a model of the functions you will need to write, and how to pull them together in one script. For this part, you have the process below 

## Getting Business IDs

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

### Open file and slice ID

1. Open data/wineries.csv
2. Slice out the 'name' and 'id' columns for each row

In [ ]:
df_saved = pd.read_csv("data/filtered_aliases.csv", index_col=0)
df_saved.reset_index(drop=True, inplace=True)
df_saved

In [ ]:
# Slice out the elements from the "name" column

df_saved_name = df_saved['name'].to_list()
df_saved_name

In [ ]:
df_saved_id = df_saved['id'].to_list()
df_saved_id

## Requesting Reviews

- Write a function that takes a business id and makes a call to the API for reviews.


### ƒ: yelp_id_offset

In [ ]:
#Testing how to create URL properly for API request

biz_id = "DknnpiG1p4OoM1maFshzXA"
url = 'https://api.yelp.com/v3/businesses/' + biz_id + '/reviews'
url

In [ ]:
# f'https /{biz_id}/reviews

In [ ]:
# for biz in biz_ids:
#     reviews = reviewcall(biz)
#     parsed_reviews = parse_reviews(reviews, biz)
    

In [ ]:
def yelp_id_offset(biz_id, yelp_key, offset=0, verbose=True):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/'+ f'{biz_id}' + '/reviews'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
#                     'id': biz_id.replace(' ', '+'),
                    'limit': 50,
                    'offset': offset
                        }
    
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
        
    return response.json()

#### Test ID Offset

In [ ]:
# print(len(reviews1['reviews']))

##### Explore Reviews1 Keys

In [ ]:
# reviews_list

In [ ]:
# print(type(reviews_list['reviews']))

In [ ]:
# reviews1['reviews'][0]

In [ ]:
# reviews1['reviews'][0].keys()

In [ ]:
# reviews1['reviews'][0]['user']

### ƒ: Parse_Reviews

In [ ]:
def parse_reviews(list_of_reviews):
    '''Adapted from Tyrell's code'''  

    # Create empty list to store results
    
    parsed_reviews = []
    
    # Loop through each review in the list of reviews
    # Add specific k:v pairs to a dictionary
    # These pairs will be used to build a DF afterwards
    
#     print('Completed: create list ' + f'{parsed_reviews}')
    
    for review in list_of_reviews:
        details = {'Reviewer Name': review['user']['name'],
            'Review ID': review['id'],
            'Time Created': review['time_created'],
            'Review Rating': review['rating'],
            'Review Text': review['text']
            }
        
        # Add the new dictionary to the empty list
        
        parsed_reviews.append(details)
    
#     print('Completed: append list')
    
    # Create a DataFrame from the resulting list
    
    df_parsed_reviews = pd.DataFrame(parsed_reviews)

#     print('Created DF')
    
    return df_parsed_reviews

In [ ]:
# df_parsed_reviews = parse_reviews(reviews1['reviews'])
# # df_parsed_reviews

### ƒ: Get_Full_Reviews

In [ ]:
def get_reviews(biz_id, yelp_key,verbose=False):
    '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    
    # Process first request to Yelp API and calculate number of pages 
    results = yelp_id_offset(biz_id, yelp_key, offset=0, verbose=False)
    num_pages = results['total']//50+1
    
    # Print out confirmation feedback
    if verbose == True:
        print(f'For {biz_id}: ')
        print(f"    Total number of results: {results['total']}.")
        print(f'    Total number of pages: {num_pages}.')
    
    # Create offset for further results and create empty list
    cur = len(results['reviews'])
    parsed_results_dfs = []

    # Retrieves remaining pages
    for num in range(1,num_pages):
        try:
            results = yelp_id_offset(biz_id, yelp_key, offset=cur, verbose=False)
            parsed_results = parse_reviews(results['reviews'])
            parsed_results['Business ID'] = biz_id
            parsed_results_dfs.append(parsed_results)
            cur += len(results['reviews'])
        except Exception as e:
            print(f'Error on page {num}.')
            print(e)

    # Concatenate DataFrames and save to .csv
    try:
        df_concat = pd.concat(parsed_results_dfs, ignore_index=True)
        df_concat.to_csv('data/business_ids.csv')
        return df_concat
    except Exception as e:
        print(f'Error, did not concat.')
        print(e)
#         return parsed_results_dfs


#     try:
#         df_concat.to_csv('data/business_ids.csv')
# #         print(f'Saved to data/business_ids.csv.')
#     except:
#         print(f'Error, did not save.')
        
#     return df_concat

In [ ]:
biz_id

In [ ]:
# results = yelp_id_offset(biz_id, yelp_key, offset=0, verbose=False)
# num_pages = 2 #results['total']//50+1

# # Print out confirmation feedback
# print(f'For {biz_id}: ')
# print(f"    Total number of results: {results['total']}.")
# print(f'    Total number of pages: {num_pages}.')

# # Create offset for further results and create empty list
# cur = 50
# parsed_results_dfs = []

# # Retrieves remaining pages
# for num in range(num_pages-1):
#     try:
#         results = yelp_id_offset(biz_id, yelp_key, offset=cur, verbose=False)
#         parsed_results = parse_reviews(results['reviews'])
#         parsed_results_dfs.append(parsed_results)
#         cur += 50
#     except Exception as e:
#         print(f'Error on page {num}.')
#         print(e)

# # Concatenate DataFrames and save to .csv
# df_concat = pd.concat(parsed_results_dfs, ignore_index=True)


In [ ]:
reviews_list = []

for biz_id in df_saved_id:
    review_result = get_reviews(biz_id, yelp_key)
    if review_result is not None:
        reviews_list.append(review_result)

reviews_list
    


In [ ]:
df_reviews = pd.concat(reviews_list, ignore_index=True)
df_reviews

## Parsing Reviews

- Write a function to parse out the relevant information from the reviews

### Adjust prior function

In [ ]:
# def parse_data(list_of_data):
#     '''Adapted from Tyrell's code'''  


#     parsed_data = []
     
#     for business in list_of_data:
#         if 'price' in business:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'price': business['price'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
#         else:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
    
#         parsed_data.append(details)
    
#     for biz in parsed_data:
#         biz['location'] = ' '.join(biz['location'])
        
#     df_parsed_data = pd.DataFrame(parsed_data)
    
# #     df_parsed_data.dropna(inplace=True)
    
#     return df_parsed_data

## Saving Parsed Data

- Write a function to save the parse data into a csv file containing all of the reviews. 

### Adjust function to save to new file

In [ ]:
# def get_full_data(term, location, yelp_key, file_name = 'data/wineries.csv'):
#     '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    
#     # Process first request to Yelp API and calculate number of pages 
#     results = yelp_request_offset(term, location, yelp_key, offset=0, verbose=False)
#     num_pages = results['total']//50+1
    
#     # Print out confirmation feedback
#     print(f'For {term} and {location}: ')
#     print(f"    Total number of results: {results['total']}.")
#     print(f'    Total number of pages: {num_pages}.')
    
#     # Create offset for further results and create empty list
#     cur = 50
#     parsed_results_dfs = []

#     # Retrieves remaining pages
#     for num in range(num_pages-1):
#         try:
#             results = yelp_request_offset(term, location, yelp_key, offset=cur, verbose=False)
#             parsed_results = parse_data(results['businesses'])
#             parsed_results_dfs.append(parsed_results)
#             cur += 50
#         except:
#             print(f'Error on page {num}.')

#     # Concatenate DataFrames and save to .csv
#     df_concat = pd.concat(parsed_results_dfs, ignore_index=True)

#     try:
#         df_concat.to_csv(file_name)
#         print(f'Saved to {file_name}.')
#     except:
#         print(f'Error, did not save.')
        
#     return df_concat

## Combining Functions

- Combine the functions above into a single script  

# Part 4 -  Using python and pandas, write code to answer the questions below. 

**Reviews**

Which are the 5 most reviewed businesses in your dataset?

What is the highest rating received in your data set and how many businesses have that rating?

What percentage of businesses have a rating greater than or  4.5?

What percentage of businesses have a rating less than 3?

---

**Pricing**

What percentage of your businesses have a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?

---

**Returing Reviews**

Return the text of the reviews for the most reviewed business. 

Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 

Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 

___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20